# Create data sets for Keras-genomics

In [1]:
libs <- c(
    'RColorBrewer',
    'ggplot2',
    'xgboost',
    'glmnet',
    'dplyr',
    'tidyr',
    'pROC',
    'stringr',
    'caret',
    'caTools',
    'RPostgreSQL',
    'RUnit',
    'igraph',
    'tibble'
)

for (lib in libs) {
        if (!require(lib, character.only = TRUE, quietly = TRUE)) {
            install.packages(lib, repos='http://cran.us.r-project.org')
        }
}

# source("http://bioconductor.org/biocLite.R")
# biocLite("BSgenome.Hsapiens.UCSC.hg38")
library(BSgenome.Hsapiens.UCSC.hg38)
hg38 = BSgenome.Hsapiens.UCSC.hg38

(.packages())

source("utility_functions.R")
source("stat_functions.R")
source("plot_functions.R")
source("~/git-repos/BDDS/trenadb/src/utils.R")

# rm(list = setdiff(ls(), lsf.str()))

Loaded glmnet 2.0-5


Attaching package: ‘dplyr’

The following object is masked from ‘package:xgboost’:

    slice

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘tidyr’

The following object is masked from ‘package:Matrix’:

    expand

Type 'citation("pROC")' for a citation.

Attaching package: ‘pROC’

The following object is masked from ‘package:glmnet’:

    auc

The following objects are masked from ‘package:stats’:

    cov, smooth, var


Attaching package: ‘igraph’

The following object is masked from ‘package:stringr’:

    %>%

The following objects are masked from ‘package:tidyr’:

    %>%, crossing

The following objects are masked from ‘package:dplyr’:

    %>%, as_data_frame, groups, union

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

[1] "BSgenome.Hsapiens.UCSC.hg38" "BSgenome"                   
 [3] "rtracklayer"                 "Biostrings"                 
 [5] "XVector"                     "GenomicRanges"              
 [7] "GenomeInfoDb"                "IRanges"                    
 [9] "S4Vectors"                   "stats4"                     
[11] "BiocGenerics"                "parallel"                   
[13] "igraph"                      "RUnit"                      
[15] "RPostgreSQL"                 "DBI"                        
[17] "caTools"                     "caret"                      
[19] "lattice"                     "stringr"                    
[21] "pROC"                        "tidyr"                      
[23] "dplyr"                       "glmnet"                     
[25] "foreach"                     "Matrix"                     
[27] "xgboost"                     "ggplot2"                    
[29] "RColorBrewer"                "stats"                      
[31] "graphics"                    "grDevices"                  
[33] "utils"                       "datasets"                   
[35] "methods"                     "base"

[1] --- test.locStringToBedTable
[1] --- test.getFimoHits
[1] --- test.getHits
[1] --- test.createHintTable
[1] found 194 hint hits in 101 bases
[1] found 16 hint hits in 101 bases
[1] found 14 hint hits in 101 bases
[1] found 10 hint hits in 19 bases
[1] found 995 hint hits in 20019 bases
[1] --- test.createHintTable_ignoreStrand
[1] found 44 hint hits in 12 bases
[1] found 44 hint hits in 12 bases
[1] eliminating 11 double-stranded hits
[1] --- test.createWellingtonTable
[1] found 126 wellington hits in 19 bases
[1] found 378 wellington hits in 20019 bases
[1] found 68 wellington hits in 12 bases
[1] found 4 wellington hits in 12 bases
[1] found 126 wellington hits in 19 bases
[1] found 64 wellington hits in 19 bases
[1] found 24 wellington hits in 19 bases
[1] --- test.createWellingtonTable_ignoreStrand, deferred
[1] found 26 wellington hits in 10 bases
[1] found 26 wellington hits in 10 bases
[1] eliminating 13 double-stranded hits
[1] --- test.createPiqTable
[1] found 342 piq hits

## Load Data (with overlaps)

In [352]:
load("train_test_valid_data_sets.Rdata")

## tasks:
- within each df:
  - pare down al IC sets to no regions within 100 bp of each other
- between dfs:
  - remove any regions in (pared down) IC within 100 bp of any regions in unpared-down EC sets

## Function for filtering within a set

In [489]:
# NOTE: this function assumes the locations are on the same chromasome

# sort centers and keep track of distance to last good center
get.ind.loc.ids.greedy.fast <- function(df.with.centers, thresh=100) {
    df.with.centers.tbl <- as_data_frame(df.with.centers)
    df.with.centers.ordered <- df.with.centers.tbl[with(df.with.centers.tbl, order(center)), ]
    
    centers <- df.with.centers.ordered$center
    locids <- df.with.centers.ordered$locid
    ind.ids <- c()
    
    for (r in 1:nrow(df.with.centers)) {
        if (length(ind.ids) == 0) {
            ind.ids <- c(ind.ids, locids[r])
            last.ind.cent <- centers[r]
        } else {
            this.cent <- centers[r]
            if (this.cent - last.ind.cent > thresh) {
                ind.ids <- c(ind.ids, locids[r])
                last.ind.cent <- this.cent
            }
        }
    }
    return(ind.ids)
}

## Filter all three IC sets together (then split again later)

In [490]:
df <- rbind(ic.train.df, ic.test.df, ic.valid.df)

In [491]:
dim(df)

[1] 568097     51

In [493]:
# add a key to the original df
df$locid <- 1:nrow(df)

# create a location df to use for the filtering, add the same key so we can merge back
locs.df <- locStringToBedTable(df$loc)
locs.df$center <- as.integer(round(0.5*(locs.df$start + locs.df$end)))
locs.df$locid <- as.integer(rownames(locs.df))
locs.df <- as_data_frame(locs.df)
locs.df <- locs.df[with(locs.df, order(locid)), ]

# get a greedy set of independent locations
ind.loc.ids <- get.ind.loc.ids.greedy.fast(locs.df)

# subset df on those ids, and drop the locid column
ind.df <- df[df$locid %in% ind.loc.ids,!(colnames(df) %in% c("locid"))]

In [494]:
dim(ind.df)

[1] 135841     51

### Check to make sure all locs are non overlapping

In [568]:
# need to add an index column (lodid) to be able to properly match things
my.locs.df <- locStringToBedTable(ind.df$loc)
my.locs.df$locid <- as.integer(rownames(my.locs.df))
my.locs.df <- as_data_frame(my.locs.df)
my.locs.df <- my.locs.df[with(my.locs.df, order(locid)), ]

my.locs.df$center <- as.integer(round(0.5*(my.locs.df$start + my.locs.df$end)))

my.chr <- "chr19"
my.gr <- GRanges(seqnames=my.chr,
                 IRanges(start=my.locs.df$center-50, end=my.locs.df$center+50))

my.overlaps.df <- as_data_frame(findOverlaps(my.gr))
my.overlaps.noself.df <- subset(my.overlaps.df, subjectHits != queryHits)

nrow(my.overlaps.noself.df)

Warning message in as.data.frame(as(x, "DataFrame"), row.names = row.names, optional = optional, :
“Arguments in '...' ignored”

[1] 0

## Remove any remaining IC locs that are overlap any EC data

In [569]:
all.ec.df <- rbind(ec.train.df, ec.test.df, ec.valid.df)

In [570]:
# need to add an index column (lodid) to be able to properly match things
ec.locs.df <- locStringToBedTable(all.ec.df$loc)
ec.locs.df$locid <- as.integer(rownames(ec.locs.df))
ec.locs.df <- as_data_frame(ec.locs.df)
ec.locs.df <- ec.locs.df[with(ec.locs.df, order(locid)), ]

ec.locs.df$center <- as.integer(round(0.5*(ec.locs.df$start + ec.locs.df$end)))

ec.chr <- "chr19"
ec.gr <- GRanges(seqnames=ec.chr,
                 IRanges(start=ec.locs.df$center-50, end=ec.locs.df$center+50))

ind.ec.overlaps.df <- as_data_frame(findOverlaps(my.gr,ec.gr))

### Check some random samples to verify we're finding correct overlaps

In [584]:
i.overlap <- sample(nrow(ind.ec.overlaps.df), 5)

i.ind <- ind.ec.overlaps.df[i.overlap,]$queryHits
i.ec <- ind.ec.overlaps.df[i.overlap,]$subjectHits

data.frame(ec.loc=all.ec.df[i.ec,]$loc,ind.loc=ind.df[i.ind,]$loc)

ec.loc,ind.loc
chr19:2353728-2353746,chr19:2353738-2353749
chr19:17007199-17007209,chr19:17007200-17007210
chr19:45767973-45767987,chr19:45768038-45768050
chr19:37894797-37894806,chr19:37894795-37894806
chr19:18657788-18657799,chr19:18657789-18657799


In [592]:
ind.to.remove <- unique(ind.ec.overlaps.df$queryHits)
ind.to.keep <- !(1:nrow(ind.df) %in% ind.to.remove)
ind.ecpruned.df <- ind.df[ind.to.keep,]

### last check: no overlaps between ind.ecpruned.df and all.ec.df

In [604]:
pruned.locs.df <- locStringToBedTable(ind.ecpruned.df$loc)
pruned.locs.df$locid <- as.integer(rownames(pruned.locs.df))
pruned.locs.df <- as_data_frame(pruned.locs.df)
pruned.locs.df <- pruned.locs.df[with(pruned.locs.df, order(locid)), ]

pruned.locs.df$center <- as.integer(round(0.5*(pruned.locs.df$start + pruned.locs.df$end)))

pruned.chr <- "chr19"
pruned.gr <- GRanges(seqnames=pruned.chr,
                 IRanges(start=pruned.locs.df$center-50, end=pruned.locs.df$center+50))

should.be.no.overlaps <- as_data_frame(findOverlaps(pruned.gr,ec.gr))
nrow(should.be.no.overlaps)

Warning message in as.data.frame(as(x, "DataFrame"), row.names = row.names, optional = optional, :
“Arguments in '...' ignored”

[1] 0

## Re-split IC data into smaller train/test/valid sets

In [606]:
split.test.train.valid <- function(df, p.tt=0.8, p.valid=0.1) {
    # p.valid is the portion of the data set aside for validation
    # p.tt is the fraction of the remaining data used for training

    require("tibble")
    
    df <- as_data_frame(df)
    N.all <- dim(df)[1]

    all.inds <- c(1:N.all)
    valid.inds <- sample(N.all, round(p.valid*N.all), replace=FALSE)

    valid.inds.bool <- all.inds %in% valid.inds
    tt.inds.bool <- !valid.inds.bool
    
    valid.df <- df[valid.inds.bool,]
    tt.df <- df[tt.inds.bool,]

    N.tt <- dim(tt.df)[1]
    tt.inds <- c(1:N.tt)
    train.inds <- sample(N.tt, round(p.tt*N.tt), replace=FALSE)

    train.inds.bool <- tt.inds %in% train.inds
    test.inds.bool <- !train.inds.bool

    train.df <- tt.df[train.inds.bool,]
    test.df <- tt.df[test.inds.bool,]

    out.list <- list()
    out.list$train <- train.df
    out.list$test <- test.df
    out.list$valid <- valid.df
    
    return(out.list)
}

In [607]:
ic.ind.pruned.train.test.valid.list <- split.test.train.valid(ind.ecpruned.df)

ic.train.df <- ic.ind.pruned.train.test.valid.list$train
ic.test.df <- ic.ind.pruned.train.test.valid.list$test
ic.valid.df <- ic.ind.pruned.train.test.valid.list$valid

In [608]:
nrow(ic.train.df)
nrow(ic.test.df)
nrow(ic.valid.df)

[1] 23562

[1] 5891

[1] 3272

## Pull 101-length seq around motifs

In [4]:
# custom function from biostars since library version hard wraps lines
writeFASTA <- function(dna,  desc=names(dna),  file=stdout()) {
  if (is.null(desc))
    desc <- paste(seq(along=dna))
  fasta = character(2 * length(dna))
  fasta[c(TRUE, FALSE)] <- paste(">", desc, sep="")
  fasta[c(FALSE, TRUE)] <- as.character(dna)
  writeLines(fasta, file)
}

In [234]:
# subsample cases to balance positive and negative samples
# and use only unique locations

make_DL_tbl <- function(Z.case, balanced=TRUE) {
    my.table <- as_data_frame(Z.case[,c("loc", "start", "end", "csscore")])
    my.table.nodupes <- my.table[!duplicated(my.table[1]),]
    if (balanced==FALSE) {
        table.to.return <- my.table.nodupes
    } else {
        my.table.nodupes.cs.yes <- my.table.nodupes[my.table.nodupes$csscore!=0,]
        my.table.nodupes.cs.no <- my.table.nodupes[my.table.nodupes$csscore==0,]
        downsampled.cs.no.inds <- sample(dim(my.table.nodupes.cs.no)[1],
                                         dim(my.table.nodupes.cs.yes)[1], replace=FALSE)
        my.table.nodupes.cs.no.downsampled <- my.table.nodupes.cs.no[downsampled.cs.no.inds,]
        table.to.return <- rbind(my.table.nodupes.cs.no.downsampled,my.table.nodupes.cs.yes)
    }
    table.to.return.shuffled <- table.to.return[sample(nrow(table.to.return),replace=FALSE),]
    return(table.to.return.shuffled)
}

In [268]:
# make traintest and validation data sets for DL
X.traintest.DL <- make_DL_tbl(Z.train)
X.valid.DL <- make_DL_tbl(Z.test)

# split traintest into train and test for DL
N.traintest.DL <- dim(X.traintest.DL)[1]
DL.traintest.inds <- c(1:N.traintest.DL)
DL.train.inds <- sample(N.traintest.DL, round(0.75*N.traintest.DL), replace=FALSE)

X.train.DL <- X.traintest.DL[DL.traintest.inds %in% DL.train.inds,]
X.test.DL <- X.traintest.DL[!(DL.traintest.inds %in% DL.train.inds),]

In [270]:
dim(X.train.DL)
dim(X.test.DL)

[1] 79407     4

[1] 26469     4

In [271]:
intersect(X.valid.DL$loc, X.train.DL$loc)

[1] "chr19:6532367-6532377"   "chr19:10422225-10422231"
   [3] "chr19:40570626-40570635" "chr19:16897251-16897261"
   [5] "chr19:41264065-41264079" "chr19:13952457-13952471"
   [7] "chr19:47274678-47274688" "chr19:5322508-5322519"  
   [9] "chr19:9675106-9675117"   "chr19:39833529-39833545"
  [11] "chr19:33373601-33373614" "chr19:11925052-11925069"
  [13] "chr19:14381198-14381205" "chr19:47130650-47130660"
  [15] "chr19:49580594-49580603" "chr19:57888806-57888815"
  [17] "chr19:2783750-2783759"   "chr19:45038258-45038269"
  [19] "chr19:16660240-16660253" "chr19:7701865-7701880"  
  [21] "chr19:39402674-39402683" "chr19:16330557-16330567"
  [23] "chr19:43575695-43575704" "chr19:10871101-10871115"
  [25] "chr19:7595713-7595723"   "chr19:16472051-16472067"
  [27] "chr19:36115685-36115695" "chr19:46714579-46714589"
  [29] "chr19:55599701-55599710" "chr19:49017944-49017954"
  [31] "chr19:12331319-12331332" "chr19:55600339-55600349"
  [33] "chr19:11197861-11197870" "chr19:14005918-14005933"
  [35] "chr19:45179156-45179169" "chr19:55156551-55156561"
  [37] "chr19:55063304-55063314" "chr19:38850321-38850335"
  [39] "chr19:48322015-48322032" "chr19:43475330-43475340"
  [41] "chr19:3285448-3285457"   "chr19:49415036-49415045"
  [43] "chr19:35694112-35694126" "chr19:9791948-9791964"  
  [45] "chr19:43475414-43475423" "chr19:6739543-6739559"  
  [47] "chr19:40690697-40690713" "chr19:10404914-10404924"
  [49] "chr19:50333518-50333532" "chr19:19172223-19172236"
  [51] "chr19:54449147-54449161" "chr19:1180540-1180554"  
  [53] "chr19:7739513-7739526"   "chr19:18571839-18571848"
  [55] "chr19:4229429-4229445"   "chr19:48322065-48322079"
  [57] "chr19:3605135-3605144"   "chr19:8050635-8050644"  
  [59] "chr19:18174465-18174474" "chr19:11482727-11482737"
  [61] "chr19:2051340-2051350"   "chr19:10106253-10106264"
  [63] "chr19:39046335-39046346" "chr19:55339755-55339772"
  [65] "chr19:19668717-19668727" "chr19:18118326-18118335"
  [67] "chr19:35205508-35205518" "chr19:1407124-1407131"  
  [69] "chr19:9140402-9140412"   "chr19:15107616-15107628"
  [71] "chr19:18205221-18205230" "chr19:33302713-33302724"
  [73] "chr19:41871674-41871680" "chr19:4867134-4867146"  
  [75] "chr19:14363717-14363727" "chr19:2627361-2627371"  
  [77] "chr19:56366580-56366589" "chr19:3762728-3762738"  
  [79] "chr19:8512213-8512222"   "chr19:4670387-4670394"  
  [81] "chr19:45769282-45769291" "chr19:45439998-45440007"
  [83] "chr19:1295909-1295918"   "chr19:45451051-45451061"
  [85] "chr19:47366160-47366170" "chr19:1863627-1863641"  
  [87] "chr19:48511850-48511866" "chr19:43584281-43584292"
  [89] "chr19:55642866-55642875" "chr19:38870145-38870158"
  [91] "chr19:38870120-38870130" "chr19:18117854-18117868"
  [93] "chr19:14433302-14433311" "chr19:56121452-56121463"
  [95] "chr19:28708959-28708968" "chr19:48511853-48511863"
  [97] "chr19:52269543-52269553" "chr19:49590198-49590215"
  [99] "chr19:1942135-1942151"   "chr19:45456010-45456017"
 [101] "chr19:17539978-17539988" "chr19:58408505-58408521"
 [103] "chr19:3180150-3180160"   "chr19:45770741-45770752"
 [105] "chr19:18340588-18340597" "chr19:55648398-55648408"
 [107] "chr19:13009088-13009095" "chr19:1260412-1260422"  
 [109] "chr19:1450015-1450024"   "chr19:19859082-19859092"
 [111] "chr19:47274355-47274365" "chr19:1099694-1099704"  
 [113] "chr19:33521825-33521835" "chr19:17470240-17470255"
 [115] "chr19:11374938-11374948" "chr19:55655579-55655592"
 [117] "chr19:2266668-2266675"   "chr19:10295654-10295664"
 [119] "chr19:19663685-19663694" "chr19:48364183-48364193"
 [121] "chr19:48468935-48468945" "chr19:10871279-10871289"
 [123] "chr19:2273898-2273907"   "chr19:39125977-39125987"
 [125] "chr19:12793893-12793907" "chr19:34134460-34134469"
 [127] "chr19:3094229-3094239"   "chr19:45485424-45485434"
 [129] "chr19:35248572-35248582" "chr19:42129449-42129463"
 [131] "chr19:17776699-17776712" "chr19:35899883-35899893"
 [133] "chr19:5130503-5130514"   "chr19:29811630-29811640"
 [135] "chr19:49085281-49085291" "chr19:48322054-4

In [223]:
string.locs.split <- strsplit(my.table.balanced.classes.shuffled$loc, ":", fixed = TRUE)
window.chrs <- sapply(string.locs.split, function(x) x[1])

motif.starts <- my.table.balanced.classes.shuffled$start
motif.stops <- my.table.balanced.classes.shuffled$end
motif.centers <- round(0.5*(motif.starts + motif.stops))

window.starts <- motif.centers - 50
window.stops <- motif.centers + 50

In [224]:
motif.windows <- getSeq(hg38, window.chrs, window.starts, window.stops)

In [123]:
deep.train.inds.end <- round(0.75*length(train.inds))
deep.train.inds <- train.inds[1:deep.train.inds.end]
deep.test.inds <- train.inds[deep.train.inds.end:length(train.inds)]

In [168]:
# write out train/test/valid sets for seep learning

window.chrs.train <- window.chrs[deep.train.inds]
window.chrs.test <- window.chrs[deep.test.inds]
window.chrs.val <- window.chrs[-train.inds]

window.starts.train <- window.starts[deep.train.inds]
window.starts.test <- window.starts[deep.test.inds]
window.starts.val <- window.starts[-train.inds]

window.stops.train <- window.stops[deep.train.inds]
window.stops.test <- window.stops[deep.test.inds]
window.stops.val <- window.stops[-train.inds]

motif.windows.train <- getSeq(hg38, window.chrs.train, window.starts.train, window.stops.train)
motif.windows.test <- getSeq(hg38, window.chrs.test, window.starts.test, window.stops.test)
motif.windows.val <- getSeq(hg38, window.chrs.val, window.starts.val, window.stops.val)

window.locs.train <- paste0(window.chrs.train, ":", window.starts.train, "-", window.stops.train)
window.locs.test <- paste0(window.chrs.test, ":", window.starts.test, "-", window.stops.test)
window.locs.val <- paste0(window.chrs.val, ":", window.starts.val, "-", window.stops.val)

motif.windows.char.train <- as.character(motif.windows.train)
motif.windows.char.test <- as.character(motif.windows.test)
motif.windows.char.val <- as.character(motif.windows.val)

names(motif.windows.char.train) <- window.locs.train
names(motif.windows.char.test) <- window.locs.test
names(motif.windows.char.val) <- window.locs.val

writeFASTA(motif.windows.char.train, file='~/train.fa')
writeFASTA(motif.windows.char.test, file='~/test.fa')
writeFASTA(motif.windows.char.val, file='~/valid.fa')

# writeXStringSet(motif.windows.train, filepath="~/train.fa")
# writeXStringSet(motif.windows.test, filepath="~/test.fa")
# writeXStringSet(motif.windows.val, filepath="~/valid.fa")

In [172]:
# write out target tsv files

y.deep.train <- y[deep.train.inds]
y.deep.test <- y[deep.test.inds]
y.deep.val <- y[-train.inds]

df.target.train <- cbind(y.deep.train, as.numeric(!y.deep.train))
df.target.test <- cbind(y.deep.test, as.numeric(!y.deep.test))
df.target.val <- cbind(y.deep.val, as.numeric(!y.deep.val))

write.table(df.target.train, file='~/train.target', quote=FALSE, sep='\t', col.names = FALSE, row.names = FALSE)
write.table(df.target.test, file='~/test.target', quote=FALSE, sep='\t', col.names = FALSE, row.names = FALSE)
write.table(df.target.val, file='~/valid.target', quote=FALSE, sep='\t', col.names = FALSE, row.names = FALSE)

In [174]:
mean(1-y.deep.val)

[1] 0.9006347